In [1]:
import csv, sys
from bs4 import BeautifulSoup
from datetime import datetime
from joblib import Parallel, delayed
import re
import pandas as pd

train_dataset_dir = './train/'
test_dataset_dir = './test/'

from os import listdir
from os.path import isfile, isdir, join


以下是loading data

In [2]:
train_df = None
test_df = None
train_files = listdir(train_dataset_dir)
test_files = listdir(test_dataset_dir)

train_df = pd.read_csv(train_dataset_dir + 'title_len.csv')
test_df = pd.read_csv(test_dataset_dir + 'title_len.csv')

In [3]:
for f in train_files:
        s = f[:-4]
        temp_df = pd.read_csv(train_dataset_dir + s + '.csv', lineterminator='\n')
        train_df[s] = temp_df[s]

In [4]:
for f in test_files:
        s = f[:-4]
        temp_df = pd.read_csv(test_dataset_dir + s + '.csv', lineterminator='\n')
        test_df[s] = temp_df[s]

In [5]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from matplotlib import pyplot as plt
import seaborn as sns

In [6]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

解釋一下下面兩個transformer，可以當成standardScalar的感覺，call fit會把丟入的trainset去算dictionary跟mean(就跟SC的fit一樣，要先看train的平均一樣)，Call transform會把input transform成有popularity的datafram，所以可以放進pipeline

In [176]:
class AuthorTransFormer(BaseEstimator,TransformerMixin):
    def __init__(self, gate = 5):
        self.author_pop_dict = {}
        self.author_pop_mean = 0
        self.gate = 5
        
    def get_author_pop(self, author_arr):
        total_mean = []
        for i in author_arr:
            if i in self.author_pop_dict.keys():
                total_mean.append(self.author_pop_dict[i])
            else:
                total_mean.append(self.author_pop_mean)
        return total_mean

    def get_train_author_pop(self, author_arr, label_arr):
        popularity = {}
        for author, label in zip(author_arr, label_arr):
            if author in popularity.keys():
                popularity[author].append(int(label))
            else:
                popularity[author] = [int(label)]

        delete_keys = []
        for i in popularity.keys():
            if(len(popularity[i]) < self.gate):
                delete_keys.append(i)
            else:
                popularity[i] = sum(popularity[i]) / len(popularity[i])
                
        for j in delete_keys:
            popularity.pop(j, None)
        total_mean = []
        cnt = 0
        for i in author_arr:
            if i in popularity.keys():
                total_mean.append(popularity[i])
            else:
                cnt = cnt + 1
                total_mean.append(-1)

        mean = (sum(total_mean) + cnt)/ (len(total_mean) - cnt)
        return popularity, mean


    def fit(self,X,y=None):
        self.author_pop_dict, self.author_pop_mean = self.get_train_author_pop(X['author'], X['label'])
        return self
    
    def transform(self,X,y=None):
        X['popularity'] = self.get_author_pop(X['author'])
        return X

In [177]:
class ChannelTransFormer(BaseEstimator,TransformerMixin):
    def __init__(self, gate = 5):
        self.author_pop_dict = {}
        self.author_pop_mean = 0
        self.gate = 5
        
    def get_author_pop(self, author_arr):
        total_mean = []
        for i in author_arr:
            if i in self.author_pop_dict.keys():
                total_mean.append(self.author_pop_dict[i])
            else:
                total_mean.append(self.author_pop_mean)
        return total_mean

    def get_train_author_pop(self, author_arr, label_arr):
        popularity = {}
        for author, label in zip(author_arr, label_arr):
            if author in popularity.keys():
                popularity[author].append(int(label))
            else:
                popularity[author] = [int(label)]

        delete_keys = []
        for i in popularity.keys():
            if(len(popularity[i]) < self.gate):
                delete_keys.append(i)
            else:
                popularity[i] = sum(popularity[i]) / len(popularity[i])
                
        for j in delete_keys:
            popularity.pop(j, None)
        total_mean = []
        cnt = 0
        for i in author_arr:
            if i in popularity.keys():
                total_mean.append(popularity[i])
            else:
                cnt = cnt + 1
                total_mean.append(-1)

        mean = (sum(total_mean) + cnt)/ (len(total_mean) - cnt)
        return popularity, mean


    def fit(self,X,y=None):
        self.author_pop_dict, self.author_pop_mean = self.get_train_author_pop(X['channel'], X['label'])
        return self
    
    def transform(self,X,y=None):
        X['channel_pop'] = self.get_author_pop(X['channel'])
        return X

In [178]:
class AuthorTopicsPopTransFormer(BaseEstimator,TransformerMixin):
    def __init__(self, gate = 5, tps_list = []):
        self.author_pop_dict = {}
        self.author_pop_mean = 0
        self.gate = 5
        self.df = None
        self.tps_list = tps_list
        
    def get_author_channel_pop(self, channel_arr, author_arr, author_pop_arr):
        arr = []
        len_arr = []
        for author, channel, author_pop in zip(author_arr, channel_arr, author_pop_arr):
            author_channel_df = self.df.loc[self.df['author'] == author]
            author_channel_df = author_channel_df.loc[author_channel_df['choose_tps'] == channel] 
            if (len(author_channel_df) < self.gate) or (channel not in self.tps_list):
                arr.append(author_pop)
                len_arr.append(len(author_channel_df))
            else:
                positive = len(author_channel_df.loc[author_channel_df['label'] == 1])
                total = len(author_channel_df)
                arr.append(positive/total)
                len_arr.append(total)
        return arr, len_arr
    
    def fit(self,X,y=None):
        self.df = X.loc[:, ['author', 'choose_tps', 'label']] 
        return self
    
    def transform(self,X,y=None):
        X['author_tps_pop'],  X['author_tps_count'] = self.get_author_channel_pop(X['choose_tps'],  X['author'],  X['popularity'])
        if 'label' in X.columns:
            X= X.drop(['author','choose_tps', 'label'], axis=1)
        else:
            X= X.drop(['author', 'choose_tps'], axis=1)
        return X

In [179]:
class TopicsTransFormer(BaseEstimator,TransformerMixin):
    def __init__(self, gate = 5, tps_list = []):
        self.topics_pop_dict = {}
        self.topics_pop_mean = 0
        self.gate = 5
        self.tps_list = tps_list
        
    def get_topics_pop(self,topics_arr):
        total_mean = []
        choose_topic = []
        for topics in topics_arr :
            tps_val = -1
            choose_tps = ''
            for tps in topics.split():
                if tps in self.topics_pop_dict.keys():
                    if tps_val < self.topics_pop_dict[tps]:
                        tps_val = self.topics_pop_dict[tps]
                        choose_tps = tps
            if topics == '' or choose_tps == '':
                total_mean.append(self.topics_pop_mean)
                choose_topic.append('NONE')
            else:
                total_mean.append(self.topics_pop_dict[choose_tps])
                choose_topic.append(choose_tps)
        return total_mean, choose_topic

    def get_train_topics_pop(self, topics_arr, label_arr):
        popularity = {}
        for i in self.tps_list:
            popularity[i] = []
        for topics, label in zip(topics_arr, label_arr):
            for tps in topics.split():
                if tps in popularity.keys():
                    popularity[tps].append(int(label))
        for i in popularity.keys():
            if len(popularity[i]) != 0:
                popularity[i] = sum(popularity[i]) / len(popularity[i])
            else: 
                print(i)
        
        total_mean = []
        cnt_non_tps = 0
        for topics in topics_arr:
            tps_val = -1
            choose_tps = ''
            for tps in topics.split():
                if tps not in popularity.keys():
                    continue
                if tps_val < popularity[tps]:
                    tps_val = popularity[tps]
                    choose_tps = tps
            if topics == '' or choose_tps == '':
                cnt_non_tps += 1
                total_mean.append(-1)
            else:
                total_mean.append(popularity[choose_tps])

        mean = (sum(total_mean) + cnt_non_tps) / (len(total_mean) - cnt_non_tps)
        return popularity, mean


    def fit(self,X,y=None):
        self.topics_pop_dict, self.topics_pop_mean = self.get_train_topics_pop(X['topics'].fillna(""), X['label'])
        return self
    
    def transform(self,X,y=None):
        X['topics_pop'] , X['choose_tps'] = self.get_topics_pop(X['topics'].fillna(""))
        X= X.drop(['topics'], axis=1)
        return X

注意，authortrans一定要擺在topic的前面，topics一定要擺在SC的前面(因為中間輸出會drop 有文字的colums)

In [188]:
class AuthorChannelPopTransFormer(BaseEstimator,TransformerMixin):
    def __init__(self, gate = 5):
        self.author_channel_pop_dict = {}
        self.author_channel_pop_mean = 0
        self.author_channel_cnt_dict = {}
        self.author_channel_cnt_mean = 0
        self.gate = 5
        self.df = None
        
    def get_author_channel_pop(self, channel_arr, author_arr):
        arr = []
        len_arr = []
        for author, channel in zip(author_arr, channel_arr):
            if (author, channel) not in self.author_channel_pop_dict.keys():
                arr.append(self.author_channel_pop_mean)
                len_arr.append(self.author_channel_cnt_mean)
            else:
                arr.append(self.author_channel_pop_dict[(author, channel)])
                len_arr.append(self.author_channel_cnt_dict[(author, channel)])
        return arr, len_arr
    
    def get_train_author_channel_pop(self, channel_arr, author_arr, label_arr):
        popularity = {}
        for author, channel, label in zip(author_arr, channel_arr, label_arr):
            if (author, channel) in popularity.keys():
                popularity[(author, channel)].append(int(label))
            else:
                popularity[(author, channel)] = [int(label)]
            

        delete_keys = []
        cnt_mean = []
        cnt_dict = {}
        for i in popularity.keys():
            if len(popularity[i]) < self.gate:
                delete_keys.append(i)
            else:
                cnt_dict[i] = len(popularity[i])
                cnt_mean.append(len(popularity[i]))
                popularity[i] = sum(popularity[i]) / len(popularity[i])
        for j in delete_keys:
            popularity.pop(j, None)

        total_mean = []
        cnt_non_author_channel = 0
        mean_cnt = 0
        for author, channel in zip(author_arr, channel_arr):
            if (author, channel) in popularity.keys():
                total_mean.append(popularity[(author, channel)])
            else:
                cnt_non_author_channel += 1
                total_mean.append(-1)
        
        mean = (sum(total_mean) + cnt_non_author_channel) / (len(total_mean) - cnt_non_author_channel)
        mean_cnt = sum(cnt_mean) / len(cnt_mean)
        return popularity, mean, cnt_dict, mean_cnt
    
    def fit(self,X,y=None):
        self.author_channel_pop_dict, self.author_channel_pop_mean, self.author_channel_cnt_dict, self.author_channel_cnt_mean = self.get_train_author_channel_pop(X['channel'], X['author'], X['label'])
        return self
    
    def transform(self,X,y=None):
        X['author_channel_pop'],  X['author_channel_count'] = self.get_author_channel_pop(X['channel'], X['author'])
        X= X.drop(['channel'], axis=1)
        return X

In [183]:
topic_dic  = ['apps', 'facebook', 'design', 'photography', 'youtube', 'google', 'twitter', 'family', 'television', 'gallery', 'movies', 'ukraine', 'search', 'thrones', 'contributor', 'glass', 'humor', 'space', 'health', 'netflix', 'wars', 'east', 'linkedin', 'climate', 'cup', 'memes', 'reddit', 'winter', 'smartphone', 'vine', 'spotify', 'holiday', 'celebrities', 'yahoo', 'gmail', 'snapchat', 'diy', 'videos', 'launchpad', 'choice', 'earth', '2014', '360', 'chrome', 'shopping', 'california', 'tablet', 'small', 'ebay', 'game', 'house', 'instagram', 'holidays', 'web', 'snow', 'streaming', 'cyrus', 'late', 'lego', 'week', 'doctor', 'motorola', 'sxsw', 'reviews', 'parenting', 'nostalgia', 'rift', 'infographics', 'pandora', "valentine's", 'freedom', 'emmys', 'doodle', 'roundup', 'reality', 'angry', 'healthcare', 'snowden', 'gifs', 'valley', 'digital', 'beer', 'mad', 'motion', 'marathon', 'security', 'cover', 'journalism', 'features', 'bill', 'gates', 'parody', 'selfie', 'sunday', 'brands', 'beauty', 'star', 'toys', 'iphone', 'interviews', 'cases', 'solutions', 'america', 'augmented', 'human', 'now', 'recent', 'ibm', 'gift', 'crash', 'tourism', 'mars', 'electronics', 'green', 'williams', 'life', 'kickstarter', 'vimeo', 'man', 'academy', 'death', 'fashion', 'texting', 'tips', 'resume', 'xbox', 'storage', 'touch', 'kids', 'olympic', 'facts', 'siri', 'spring', 'at&t', 'hbo', 'forum', 'kingdom', 'nest', 'south', 'arrested', 'iwatch', 'los', 'anonymous', 'germany', 'opinion', 'use', 'scrivan', 'european', 'email', 'travel', 'venezuela', 'smartwatch', 'moon', 'international', 'police', 'jerry', 'costumes', 'history', 'psychology', 'congress', 'cyberbullying', 'pope', 'awards', 'lawsuit', 'hacks', 'study', 'drone', 'smart', 'easter', 'ferguson', 'ted', 'width', 'cbs', 'dna', 'management', 'teens', 'olympics', 'safety', 'stem', 'yelp', 'buzzwords', 'superheroes', 'legos', 'brazil', 'cable', 'disease', 'campaign', 'paris', 'perry', 'battery', 'crafts', 'gold', 'new', 'printers', 'iron', 'bad', 'pew', 'crime', 'news', 'online', 'korea', 'art', 'extreme', 'waze', 'fox', 'dogs', 'texas', 'retina', 'seinfeld', 'hangouts', 's5', 'cute', 'box', 'future', 'wikipedia', 'team', 'animated', 'jose', 'mashups', 'living', 'ocean', 'productivity', 'true', 'voice', 'speech', 'sam', 'fov', 'password', 'thanksgiving', 'seo', 'beat', 'deaths', 'internet', 'hashtags', 'conan', 'franco', 'smartwatches', 'philippines', 'graph', 'shazam', 'recalls', 'cameras', 'hubble', 'content', 'law', 'muppets', 'wireless', 'alibaba', 'fiber', 'browser', 'metrics', 'zelda', 'oliver', 'students', 'station', 'watson', 'printing', 'rss']

In [72]:
from xgboost import XGBClassifier
pipe1 = Pipeline([('author', AuthorTransFormer()),
                  ('channel', ChannelTransFormer()),
                  ('author_channel', AuthorChannelPopTransFormer()),
                  ('topics', TopicsTransFormer()),
                  ('std', StandardScaler()),
                  ("pca", PCA()),
                  ('clf', XGBClassifier(max_depth=5,learning_rate=0.015,objective="binary:logistic", eval_metric="error"))])


注意選擇需要的feature

In [187]:
X_train = train_df[['dayhour', 'author_post', 'nn_percentage', 'rb_percentage', 'jj_percentage', 'links', 'popularity', 'article_len', 'vb_percentage', 'topics', 'channel', 'author', 'label']]
X_test = test_df[['dayhour', 'author_post', 'nn_percentage', 'rb_percentage', 'jj_percentage', 'links', 'popularity', 'article_len', 'vb_percentage', 'topics', 'channel', 'author']]
X_train, X_val, y_train, y_val = train_test_split(X_train, train_df['label'], test_size=0.33, random_state=42)

In [74]:
pipe1.fit(X_train, y_train)
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_val, pipe1.predict_proba(X_val)[:,1])
print(score)

D:\Program_files\Anaconda 3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.5275715569098418


In [214]:
from xgboost import XGBClassifier
pipe1 = Pipeline([('author', AuthorTransFormer(gate = 2)),
                  ('channel', ChannelTransFormer(gate = 2)),
                  ('author_channel', AuthorChannelPopTransFormer(gate = 2)),
                  ('topics', TopicsTransFormer(tps_list = topic_dic)),
                  ('topics_author', AuthorTopicsPopTransFormer(tps_list = topic_dic)),
                  ('std', StandardScaler()),
                  ("pca", PCA()),
                  ('clf', XGBClassifier(max_depth=5,learning_rate=0.001,objective="binary:logistic", eval_metric="error"))])

pipe1.fit(X_t, y_train)
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_val, pipe1.predict_proba(X_v)[:,1])
print(score)

D:\Program_files\Anaconda 3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.544862320638823


In [213]:
from xgboost import XGBClassifier
pipe1 = Pipeline([('author', AuthorTransFormer(gate = 2)),
                  ('channel', ChannelTransFormer(gate = 2)),
                  ('author_channel', AuthorChannelPopTransFormer(gate = 2)),
                  ('topics', TopicsTransFormer(tps_list = topic_dic)),
                  ('topics_author', AuthorTopicsPopTransFormer(tps_list = topic_dic)),
                  ('std', StandardScaler()),
                  ('clf', XGBClassifier(max_depth=5,learning_rate=0.001,objective="binary:logistic", eval_metric="error"))])

pipe1.fit(X_t, y_train)
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_val, pipe1.predict_proba(X_v)[:,1])
print(score)

D:\Program_files\Anaconda 3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.5434609757786665


In [212]:
X_t = AuthorProblem(X_train, 'train')
y_train = X_t['label']
X_v = AuthorProblem(X_val, 'test')

In [204]:
len(X_v)

9123

In [42]:
import re
from bs4 import BeautifulSoup
def preprocessor(text):
    text = BeautifulSoup(text, 'html.parser').get_text()
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text

def tokenizer(text):
    return re.split('\s+', text.strip())

def tokenizer_stem(text):
    porter = PorterStemmer()
    return [porter.stem(word) for word in re.split('\s+', text.strip())]

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
stop = stopwords.words('english')

def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\stanl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [216]:
train_df['h2'] = train_df['h2'].fillna('')
test_df['h2'] = test_df['h2'].fillna('')
test_df['article'] = test_df['article'].fillna('')
train_df['author'] = train_df['author'].fillna('')
test_df['author'] = test_df['author'].fillna('')
train_df['title'] = train_df['title'].fillna('')
test_df['title'] = test_df['title'].fillna('')
train_df['h1'] = train_df['h1'].fillna('') + train_df['author']
test_df['h1'] = test_df['h1'].fillna('') + test_df['author']
train_df['topics'] = train_df['topics'].fillna('')
test_df['topics'] = test_df['topics'].fillna('')

In [223]:
# TF-IDF Fit
tfidf_Xtrain, tfidf_Xval, y_train, y_val = train_test_split(train_df, train_df['label'], test_size=0.3, random_state=30)
doc_words = pd.concat([tfidf_Xtrain['article'], tfidf_Xtrain['h1'], tfidf_Xtrain['title'], tfidf_Xtrain['h2']])
tfidf = TfidfVectorizer(preprocessor=preprocessor, tokenizer=tokenizer_stem_nostop)
tfidf_channel = TfidfVectorizer(preprocessor=preprocessor, tokenizer=tokenizer_stem_nostop)
tfidf_channel_topics = TfidfVectorizer(preprocessor=preprocessor, tokenizer=tokenizer_stem_nostop)
tfidf.fit(doc_words)
tfidf_channel.fit(tfidf_Xtrain['channel'])
tfidf_channel_topics.fit(pd.concat([tfidf_Xtrain['channel'], tfidf_Xtrain['topics']]))

D:\Program_files\Anaconda 3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
D:\Program_files\Anaconda 3\lib\site-packages\bs4\__init__.py:343: MarkupResemblesLocatorWarning: "music" looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning


TfidfVectorizer(preprocessor=<function preprocessor at 0x0000027700BB13A8>,
                tokenizer=<function tokenizer_stem_nostop at 0x000002770C289A68>)

In [120]:
train_df = train_df.drop(columns=['article', 'h1', 'h2'])
test_df = test_df.drop(columns=['article', 'h1', 'h2'])
print(train_df)

          id  title_len  article_len                        author  \
0          0          8          577               Clara Moskowitz   
1          1         12          305              Christina Warren   
2          2         12         1114                     Sam Laird   
3          3          5          278                     Sam Laird   
4          4         10         1370               Connor Finnegan   
...      ...        ...          ...                           ...   
27638  27638          9          303  Lorenzo FranceschiBicchierai   
27639  27639          9          317                Adario Strange   
27640  27640          8          170            Christine Erickson   
27641  27641          8          430                Seth Fiegerman   
27642  27642         10          472                  Megan Ranney   

       author_post         channel  cnt_iframe  cnt_img  cnt_vdo  dayhour  \
0               18           world           0        2        0       15   
1    

In [128]:
for i in train_sparse:
    train_df = pd.concat([train_df, pd.DataFrame(i.toarray())], axis=1)

for i in test_sparse:
    test_df = pd.concat([test_df, pd.DataFrame(i.toarray())], axis=1)
#print(train_df.columns)
X_train = train_df.drop(columns=['id', 'author_post', 'cnt_iframe', 'cnt_img', 'cnt_vdo', 'topics_pop', 'topics_pop_10', 'topics_pop_15', 'topics_pop_20', 'topics_pop_25', 'topics_pop_5', 'title'])
X_test = test_df.drop(columns=['id', 'author_post', 'cnt_iframe', 'cnt_img', 'cnt_vdo', 'topics_pop', 'topics_pop_10', 'topics_pop_15', 'topics_pop_20', 'topics_pop_25', 'topics_pop_5', 'title'])
X_train, X_val, y_train, y_val = train_test_split(X_train, train_df['label'], test_size=0.33, random_state=42)
print(X_train.columns)

MemoryError: Unable to allocate 28.1 GiB for an array with shape (27643, 136450) and data type float64

In [43]:
import scipy as sp
from sklearn.feature_extraction.text import HashingVectorizer
# hash words to 1024 buckets
hashvec = HashingVectorizer(n_features=2**14,
                            preprocessor=preprocessor,
                            tokenizer=tokenizer_stem_nostop)

# no .fit needed for HashingVectorizer, since it's defined by the hash function

# transform sentences to vectors of dimension 1024
train_h1_hash = hashvec.transform(train_df['h1'])
train_h2_hash = hashvec.transform(train_df['h2'])
train_article_hash = hashvec.transform(train_df['article'])

In [166]:
train_channel_hash = hashvec.transform(train_df['channel'])
train_con = pd.concat([train_con, pd.DataFrame(train_channel_hash.toarray())], axis=1)
X_train = train_con.drop(columns=['id', 'author_post', 'cnt_iframe', 'cnt_img', 'cnt_vdo', 'topics_pop', 'topics_pop_10', 'topics_pop_15', 'topics_pop_20', 'topics_pop_25', 'topics_pop_5', 'title'])
#X_test = test_df.drop(columns=['id', 'author_post', 'cnt_iframe', 'cnt_img', 'cnt_vdo', 'topics_pop', 'topics_pop_10', 'topics_pop_15', 'topics_pop_20', 'topics_pop_25', 'topics_pop_5', 'title'])
X_train, X_val, y_train, y_val = train_test_split(X_train, train_df['label'], test_size=0.33, random_state=42)

In [149]:
train_con = pd.concat([train_df, pd.DataFrame(train_h1_hash.toarray())], axis=1)
train_con = pd.concat([train_con, pd.DataFrame(train_h2_hash.toarray())], axis=1)
train_con = pd.concat([train_con, pd.DataFrame(train_article_hash.toarray())], axis=1)
print(train_con.columns)
train_con = train_con.drop(columns=['article', 'h1', 'h2'])
X_train = train_con.drop(columns=['id', 'author_post', 'cnt_iframe', 'cnt_img', 'cnt_vdo', 'topics_pop', 'topics_pop_10', 'topics_pop_15', 'topics_pop_20', 'topics_pop_25', 'topics_pop_5', 'title'])
#X_test = test_df.drop(columns=['id', 'author_post', 'cnt_iframe', 'cnt_img', 'cnt_vdo', 'topics_pop', 'topics_pop_10', 'topics_pop_15', 'topics_pop_20', 'topics_pop_25', 'topics_pop_5', 'title'])
X_train, X_val, y_train, y_val = train_test_split(X_train, train_df['label'], test_size=0.33, random_state=42)
print(X_train.columns)

Index([         'id',   'title_len', 'article_len',      'author',
       'author_post',     'channel',  'cnt_iframe',     'cnt_img',
           'cnt_vdo',     'dayhour',
       ...
                1014,          1015,          1016,          1017,
                1018,          1019,          1020,          1021,
                1022,          1023],
      dtype='object', length=3098)
Index([    'title_len',   'article_len',        'author',       'channel',
             'dayhour', 'jj_percentage',         'label',         'links',
       'nn_percentage',    'popularity',
       ...
                  1014,            1015,            1016,            1017,
                  1018,            1019,            1020,            1021,
                  1022,            1023],
      dtype='object', length=3086)


In [156]:
test_h1_hash = hashvec.transform(test_df['h1'])
test_h2_hash = hashvec.transform(test_df['h2'])
test_article_hash = hashvec.transform(test_df['article'])
test_con = pd.concat([test_df, pd.DataFrame(test_h1_hash.toarray())], axis=1)
test_con = pd.concat([test_con, pd.DataFrame(test_h2_hash.toarray())], axis=1)
test_con = pd.concat([test_con, pd.DataFrame(test_article_hash.toarray())], axis=1)
print(test_con.columns)
test_con = test_con.drop(columns=['article', 'h1', 'h2'])
X_test = test_con.drop(columns=['id', 'author_post', 'cnt_iframe', 'cnt_img', 'cnt_vdo', 'topics_pop', 'topics_pop_10', 'topics_pop_15', 'topics_pop_20', 'topics_pop_25', 'topics_pop_5', 'title'])

Index([         'id',   'title_len',     'article', 'article_len',
            'author', 'author_post',     'channel',  'cnt_iframe',
           'cnt_img',     'cnt_vdo',
       ...
                1014,          1015,          1016,          1017,
                1018,          1019,          1020,          1021,
                1022,          1023],
      dtype='object', length=3100)


In [150]:
from xgboost import XGBClassifier
pipe1 = Pipeline([('author', AuthorTransFormer()),
                  ('channel', ChannelTransFormer()),
                  ('author_channel', AuthorChannelPopTransFormer()),
                  ('topics', TopicsTransFormer()),
                  ('std', StandardScaler()),
                  ("pca", PCA()),
                  ('clf', XGBClassifier(max_depth=5,learning_rate=0.015,objective="binary:logistic", eval_metric="error"))])

pipe1.fit(X_train, y_train)
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_val, pipe1.predict_proba(X_val)[:,1])
print(score)

D:\Program_files\Anaconda 3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.5317380312062959


In [151]:
from xgboost import XGBClassifier
pipe1 = Pipeline([('author', AuthorTransFormer()),
                  ('channel', ChannelTransFormer()),
                  ('author_channel', AuthorChannelPopTransFormer()),
                  ('topics', TopicsTransFormer()),
                  ('std', StandardScaler()),
                  ('clf', XGBClassifier(max_depth=5,learning_rate=0.015,objective="binary:logistic", eval_metric="error"))])

pipe1.fit(X_train, y_train)
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_val, pipe1.predict_proba(X_val)[:,1])
print(score)

D:\Program_files\Anaconda 3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.5410396600048862


In [152]:
from xgboost import XGBClassifier
pipe1 = Pipeline([('author', AuthorTransFormer()),
                  ('channel', ChannelTransFormer()),
                  ('author_channel', AuthorChannelPopTransFormer()),
                  ('topics', TopicsTransFormer()),
                  ('std', StandardScaler()),
                  ('clf', XGBClassifier(max_depth=5,learning_rate=0.05,objective="binary:logistic", eval_metric="error"))])

pipe1.fit(X_train, y_train)
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_val, pipe1.predict_proba(X_val)[:,1])
print(score)

D:\Program_files\Anaconda 3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.5451525591971237


In [167]:
from xgboost import XGBClassifier
pipech = Pipeline([('author', AuthorTransFormer()),
                  ('channel', ChannelTransFormer()),
                  ('author_channel', AuthorChannelPopTransFormer()),
                  ('topics', TopicsTransFormer()),
                  ('std', StandardScaler()),
                  ('clf', XGBClassifier(max_depth=5,learning_rate=0.05,objective="binary:logistic", eval_metric="error"))])

pipech.fit(X_train, y_train)
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_val, pipech.predict_proba(X_val)[:,1])
print(score)

D:\Program_files\Anaconda 3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.5451525591971237


In [153]:
print(pipe1.predict_proba(X_val)[:,1])

[0.44722664 0.3861879  0.53726065 ... 0.44468287 0.6237414  0.6583281 ]


In [158]:
X_total = train_con.drop(columns=['id', 'author_post', 'cnt_iframe', 'cnt_img', 'cnt_vdo', 'topics_pop', 'topics_pop_10', 'topics_pop_15', 'topics_pop_20', 'topics_pop_25', 'topics_pop_5', 'title'])
pipetotal = Pipeline([('author', AuthorTransFormer()),
                      ('channel', ChannelTransFormer()),
                      ('author_channel', AuthorChannelPopTransFormer()),
                      ('topics', TopicsTransFormer()),
                      ('std', StandardScaler()),
                      ('clf', XGBClassifier(max_depth=5,learning_rate=0.05,objective="binary:logistic", eval_metric="error"))])

pipetotal.fit(X_total, train_df['label'])

D:\Program_files\Anaconda 3\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Program_files\Anaconda 3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Pipeline(steps=[('author', AuthorTransFormer()),
                ('channel', ChannelTransFormer()),
                ('topics', TopicsTransFormer()), ('std', StandardScaler()),
                ('clf',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eval_metric='error', gamma=0,
                               gpu_id=-1, importance_type='gain',
                               interaction_constraints='', learning_rate=0.05,
                               max_delta_step=0, max_depth=5,
                               min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=8, num_parallel_tree=1, random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               subsample=1, tree_method='exact',
              

In [159]:
result = pipetotal.predict_proba(X_test)
result = result.round(1)
print(result[:,1])

D:\Program_files\Anaconda 3\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[0.4 0.5 0.4 ... 0.6 0.4 0.7]


In [160]:
submit_ans = pd.DataFrame()
submit_ans['Id'] = test_df['id']
submit_ans['Popularity'] = result[:,1]

print(submit_ans)
submit_ans.to_csv('./out.csv', index=0)

          Id  Popularity
0      27643         0.4
1      27644         0.5
2      27645         0.4
3      27646         0.4
4      27647         0.4
...      ...         ...
11842  39485         0.8
11843  39486         0.5
11844  39487         0.6
11845  39488         0.4
11846  39489         0.7

[11847 rows x 2 columns]


In [173]:
pd.DataFrame.sparse.from_spmatrix(train_channel_hash)

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27640,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27641,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
doc_words = pd.concat([train_df['channel'], train_df['h1']])
tfidf = TfidfVectorizer(preprocessor=preprocessor, tokenizer=tokenizer_stem_nostop)
tfidf.fit(doc_words)

TfidfVectorizer(preprocessor=<function preprocessor at 0x0000023738CFAA68>,
                tokenizer=<function tokenizer_stem_nostop at 0x000002372372FE58>)

In [16]:
train_con = pd.concat([train_df, pd.DataFrame(tfidf_channel.transform(train_df['channel']).toarray())], axis=1)
train_con = pd.concat([train_con, pd.DataFrame(tfidf.transform(train_df['h1']).toarray())], axis=1)
train_con = pd.concat([train_con, pd.DataFrame.sparse.from_spmatrix(train_h2_hash)], axis=1)
train_con = pd.concat([train_con, pd.DataFrame.sparse.from_spmatrix(train_article_hash)], axis=1)
train_con = train_con.drop(columns=['article', 'h1', 'h2'])
X_train = train_con.drop(columns=['id', 'author_post', 'cnt_iframe', 'cnt_img', 'cnt_vdo', 'topics_pop', 'topics_pop_10', 'topics_pop_15', 'topics_pop_20', 'topics_pop_25', 'topics_pop_5', 'title'])
#X_test = test_df.drop(columns=['id', 'author_post', 'cnt_iframe', 'cnt_img', 'cnt_vdo', 'topics_pop', 'topics_pop_10', 'topics_pop_15', 'topics_pop_20', 'topics_pop_25', 'topics_pop_5', 'title'])
X_train, X_val, y_train, y_val = train_test_split(X_train, train_df['label'], test_size=0.33, random_state=42)

NameError: name 'train_test_split' is not defined

In [176]:
pipenew = Pipeline([('author', AuthorTransFormer()),
                  ('channel', ChannelTransFormer()),
                  ('author_channel', AuthorChannelPopTransFormer()),
                  ('topics', TopicsTransFormer()),
                  ('std', StandardScaler()),
                  ('clf', XGBClassifier(max_depth=5,learning_rate=0.05,objective="binary:logistic", eval_metric="error"))])

pipenew.fit(X_train, y_train)
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_val, pipenew.predict_proba(X_val)[:,1])
print(score)

D:\Program_files\Anaconda 3\lib\site-packages\sklearn\utils\validation.py:509: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."
D:\Program_files\Anaconda 3\lib\site-packages\sklearn\utils\validation.py:509: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."
D:\Program_files\Anaconda 3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
D:\Program_files\Anaconda 3\lib\site-packages\sklearn\utils\validation.py:509: UserWarni

0.5456548859148453


In [177]:
pipenew2 = Pipeline([('author', AuthorTransFormer()),
                  ('channel', ChannelTransFormer()),
                  ('author_channel', AuthorChannelPopTransFormer()),
                  ('topics', TopicsTransFormer()),
                  ('std', StandardScaler()),
                  ('clf', XGBClassifier(max_depth=7,learning_rate=0.015,objective="binary:logistic", eval_metric="error"))])

pipenew2.fit(X_train, y_train)
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_val, pipenew2.predict_proba(X_val)[:,1])
print(score)

D:\Program_files\Anaconda 3\lib\site-packages\sklearn\utils\validation.py:509: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."
D:\Program_files\Anaconda 3\lib\site-packages\sklearn\utils\validation.py:509: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."
D:\Program_files\Anaconda 3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
D:\Program_files\Anaconda 3\lib\site-packages\sklearn\utils\validation.py:509: UserWarni

0.5403270726447634


In [117]:
from xgboost import XGBClassifier
from sklearn.svm import SVC
print(X_train.columns)
#X_train, X_val, y_train, y_val = train_test_split(X_train, train_df['label'], test_size=0.33, random_state=42)
pipenew2 = Pipeline([('author', AuthorTransFormer()),
                  ('channel', ChannelTransFormer()),
                  ('author_channel', AuthorChannelPopTransFormer()),
                  ('topics', TopicsTransFormer()),
                  ('std', StandardScaler()),
                  ('clf', SVC(kernel='poly',probability=True))])

pipenew2.fit(X_train, y_train)

from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_val, pipenew2.predict_proba(X_val)[:,1])
print(X_train.columns)
print(score)

Index(['dayhour', 'author_post', 'nn_percentage', 'rb_percentage',
       'jj_percentage', 'links', 'popularity', 'article_len', 'vb_percentage',
       'topics', 'channel', 'author', 'label', 'channel_pop',
       'author_channel_pop', 'author_channel_count'],
      dtype='object')
Index(['dayhour', 'author_post', 'nn_percentage', 'rb_percentage',
       'jj_percentage', 'links', 'popularity', 'article_len', 'vb_percentage',
       'topics', 'channel', 'author', 'label', 'channel_pop',
       'author_channel_pop', 'author_channel_count'],
      dtype='object')
0.5277589735894935


In [224]:
# TF-IDF Transform
text_feature = ['article', 'h1', 'h2']
train_sparse = []
test_sparse = []
for i in text_feature:
    train_sparse.append(tfidf.transform(tfidf_Xtrain[i]))
    test_sparse.append(tfidf.transform(tfidf_Xval[i]))
train_sparse.append(tfidf_channel.transform(tfidf_Xtrain['channel']))
test_sparse.append(tfidf_channel.transform(tfidf_Xval['channel']))
train_sparse.append(tfidf_channel_topics.transform(tfidf_Xtrain['channel']))
test_sparse.append(tfidf_channel_topics.transform(tfidf_Xval['channel']))
train_sparse.append(tfidf_channel_topics.transform(tfidf_Xtrain['topics']))
test_sparse.append(tfidf_channel_topics.transform(tfidf_Xval['topics']))

NameError: name 'testfidf_Xvalt_df' is not defined

In [227]:
test_sparse.append(tfidf_channel_topics.transform(tfidf_Xval['channel']))
train_sparse.append(tfidf_channel_topics.transform(tfidf_Xtrain['topics']))
test_sparse.append(tfidf_channel_topics.transform(tfidf_Xval['topics']))

In [231]:
from scipy import sparse
train_sparsematrix = sparse.hstack((train_sparse[0], train_sparse[1]))
train_sparsematrix = sparse.hstack((train_sparsematrix, train_sparse[2]))
train_sparsematrix2 = sparse.hstack((train_sparsematrix, train_sparse[4]))
train_sparsematrix2 = sparse.hstack((train_sparsematrix2, train_sparse[5]))
train_sparsematrix = sparse.hstack((train_sparsematrix, train_sparse[3]))
train_sparse[0].shape
train_sparsematrix.shape
train_sparsematrix2.shape
test_sparsematrix = sparse.hstack((test_sparse[0], test_sparse[1]))
test_sparsematrix = sparse.hstack((test_sparsematrix, test_sparse[2]))
test_sparsematrix2 = sparse.hstack((test_sparsematrix, test_sparse[4]))
test_sparsematrix2 = sparse.hstack((test_sparsematrix2, test_sparse[5]))
test_sparsematrix = sparse.hstack((test_sparsematrix, test_sparse[3]))

In [232]:
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, VotingClassifier
bag = BaggingClassifier(base_estimator=MultinomialNB(), n_estimators=500, 
                        max_samples=0.8, bootstrap=True,
                        max_features=1.0, bootstrap_features=False, 
                        n_jobs=1)
bag.fit(train_sparsematrix, y_train)
score = roc_auc_score(y_val, bag.predict_proba(test_sparsematrix)[:,1])
print(score)

0.5510908936423801


In [233]:
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, VotingClassifier
bag2 = BaggingClassifier(base_estimator=MultinomialNB(), n_estimators=500, 
                        max_samples=0.8, bootstrap=True,
                        max_features=1.0, bootstrap_features=False, 
                        n_jobs=1, random_state=1)
bag2.fit(train_sparsematrix2, y_train)
score = roc_auc_score(y_val, bag2.predict_proba(test_sparsematrix2)[:,1])
print(score)

0.5548182639281716


In [ ]:
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, VotingClassifier
bag3 = BaggingClassifier(base_estimator=MultinomialNB(), n_estimators=2000, 
                        max_samples=0.8, bootstrap=True,
                        max_features=1.0, bootstrap_features=False, 
                        n_jobs=1, random_state=1)
bag3.fit(train_sparsematrix2, y_train)
score = roc_auc_score(y_val, bag3.predict_proba(test_sparsematrix2)[:,1])
print(score)

In [44]:
bag.predict_proba(X_val_sparse)[:,1]

array([0.8934907 , 0.4785735 , 0.54578213, ..., 0.3985051 , 0.25326268,
       0.59861693])

In [33]:
from sklearn.naive_bayes import MultinomialNB
NBarticle = MultinomialNB()
NBarticle.fit(X_train_sparse0, y_train)
score = roc_auc_score(y_val, NBarticle.predict_proba(X_val_sparse0)[:,1])
print(score)

0.5410403092543539


In [34]:
NBh1 = MultinomialNB()
NBh1.fit(X_train_sparse1, y_train)
score = roc_auc_score(y_val, NBh1.predict_proba(X_val_sparse1)[:,1])
print(score)

0.5346228149148473


In [35]:
NBh2 = MultinomialNB()
NBh2.fit(X_train_sparse2, y_train)
score = roc_auc_score(y_val, NBh2.predict_proba(X_val_sparse2)[:,1])
print(score)

0.5078712119900506


In [36]:
NBchannel = MultinomialNB()
NBchannel.fit(X_train_sparse3, y_train)
score = roc_auc_score(y_val, NBchannel.predict_proba(X_val_sparse3)[:,1])
print(score)

0.5245936900646555


In [ ]:
from sklearn.ensemble import VotingClassifier
import itertools
print('[Voting]')
best_vt, best_w, best_score = None, (), -1
for a, b, c, d in list(itertools.permutations(range(0,4))):
    clf = VotingClassifier(estimators=[('article', NBarticle), ('h1', NBh1), ('h2', NBh2), ('channel', NBchannel)],
                            voting='soft', weights=[a,b,c,d])
    clf.fit()

In [41]:
X_train_sparse, X_val_sparse, y_train, y_val = train_test_split(train_sparsematrix, train_df['label'], test_size=0.33, random_state=42)

In [48]:
def ensembleCV(model1, model2, alpha, val_df, X_val_other, X_val_text):
    max = 0
    maxpick = 0
    l = []
    for a in alpha:
        pb_list = []
        P_1 = model1.predict_proba(X_val_other)[:,1]
        P_2 = model2.predict_proba(X_val_text)[:,1]
        for d in range(len(val_df)):
            p1 = P_1[d]
            p2 = P_2[d]
            pb_list.append(p1*a + p2*(1-a))
        score = roc_auc_score(val_df, pb_list) 
        if(score > max): 
            max = score 
            maxpick = a
    return(max, maxpick)

In [50]:
ans = ensembleCV(pipenew2, bag, [(i / 100) for i in range(0, 100, 10)], y_val, X_val, X_val_sparse)
print(ans)

D:\Program_files\Anaconda 3\lib\site-packages\ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Program_files\Anaconda 3\lib\site-packages\sklearn\utils\validation.py:509: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."
D:\Program_files\Anaconda 3\lib\site-packages\ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Program_files\Anaconda 3\lib\site-packages\sklear

(0.5509718543141906, 0.5)


In [52]:
ans = ensembleCV(pipenew2, bag, [(i / 200) for i in range(90, 111, 1)], y_val, X_val, X_val_sparse)
print(ans)

D:\Program_files\Anaconda 3\lib\site-packages\ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Program_files\Anaconda 3\lib\site-packages\sklearn\utils\validation.py:509: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."
D:\Program_files\Anaconda 3\lib\site-packages\ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Program_files\Anaconda 3\lib\site-packages\sklear

D:\Program_files\Anaconda 3\lib\site-packages\ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Program_files\Anaconda 3\lib\site-packages\sklearn\utils\validation.py:509: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."
D:\Program_files\Anaconda 3\lib\site-packages\ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Program_files\Anaconda 3\lib\site-packages\sklear

(0.5510571705034906, 0.46)


In [40]:
print(X_train_sparse[0])

NameError: name 'X_train_sparse' is not defined

In [44]:
train_channel_hash = hashvec.transform(train_df['channel'])
from scipy import sparse
train_sparsematrix = sparse.hstack((train_h1_hash, train_channel_hash))
train_sparsematrix = sparse.hstack((train_sparsematrix, train_h2_hash))
train_sparsematrix = sparse.hstack((train_sparsematrix, train_article_hash))
X_train_sparse, X_val_sparse, y_train, y_val = train_test_split(train_sparsematrix, train_df['label'], test_size=0.33, random_state=42)
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, VotingClassifier
from sklearn.naive_bayes import MultinomialNB
bag = BaggingClassifier(base_estimator=MultinomialNB(), n_estimators=500, 
                        max_samples=0.7, bootstrap=True,
                        max_features=1.0, bootstrap_features=False, 
                        n_jobs=1, random_state=1)
bag.fit(X_train_sparse, y_train)
score = roc_auc_score(y_val, bag.predict_proba(X_val_sparse)[:,1])
print(score)

D:\Program_files\Anaconda 3\lib\site-packages\bs4\__init__.py:343: MarkupResemblesLocatorWarning: "music" looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning


ValueError: Negative values in data passed to MultinomialNB (input X)

In [52]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MaxAbsScaler
SC = MaxAbsScaler()
SC.fit(X_train_sparse)
X_std = SC.transform(X_train_sparse)
X_val_std = SC.transform(X_val_sparse)
bag = BaggingClassifier(base_estimator=MultinomialNB(), n_estimators=500, 
                        max_samples=0.7, bootstrap=True,
                        max_features=1.0, bootstrap_features=False, 
                        n_jobs=1, random_state=1)
bag.fit(X_std, y_train)
score = roc_auc_score(y_val, bag.predict_proba(X_val_std)[:,1])
print(score)

ValueError: Negative values in data passed to MultinomialNB (input X)

In [211]:
def AuthorProblem(DF, mode):
    if mode == 'train':
        new_df = pd.DataFrame([], columns = DF.columns)
        for index, row in DF.iterrows(): 
            morethanone = False
            for i in range(len(row['author'].split())):
                if(row['author'].find(',')):
                    a = row['author'].find(',')
                    row['author'] = row['author'][:a] + " " + row['author'][a:]
                if row['author'].split()[i] == 'and' or row['author'].split()[i] == '&' or row['author'].split()[i] == ',':
                    morethanone = True
                    a = row.copy(deep = True)
                    if(i + 2 < len(row['author'].split())):
                        a.loc['author'] = row['author'].split()[i+1] + ' ' + row['author'].split()[i+2]
                        new_df = new_df.append(a)
        DF = pd.concat([DF, new_df], ignore_index=True)
    elif mode == 'test':
        for index, row in DF.iterrows():
            morethanone = False
            for i in range(len(row['author'].split())):
                if(row['author'].find(',')):
                    a = row['author'].find(',')
                    row['author'] = row['author'][:a] + " " + row['author'][a:]
                if row['author'].split()[i] == 'and' or row['author'].split()[i] == '&' or row['author'].split()[i] == ',':
                    morethanone = True
                    break
            if(morethanone):
                if len(row['author'].split()) > 1:
                    DF.loc[index,'author'] = row['author'].split()[0] + ' ' + row['author'].split()[1]
    return DF

In [148]:
pd.DataFrame([], columns = train_df.columns).shape[0]

0

In [149]:
for index, row in train_df.iterrows():
    print(index)
    print(row['author'])
    break

0
Clara Moskowitz


In [166]:
trydf = train_df.head(5)
trydf.loc[1,'author'] = 'Sam ham, Sam Sam & Ham Ham'
trydf = AuthorProblem(trydf, 'test')
print(trydf)

   id  title_len                                            article  \
0   0          8  There may be killer asteroids headed for Earth...   
1   1         12  Google took a stand of sorts against patent-la...   
2   2         12  You've spend countless hours training to be an...   
3   3          5  Tired of the same old sports fails and news fa...   
4   4         10  At 6-foot-5 and 298 pounds, All-Pro NFL star J...   

   article_len           author  author_post        channel  cnt_iframe  \
0          577  Clara Moskowitz           18          world           0   
1          305          Sam ham          399           tech           0   
2         1114        Sam Laird         1004  entertainment          25   
3          278        Sam Laird         1004    watercooler          21   
4         1370  Connor Finnegan           87  entertainment           1   

   cnt_img  cnt_vdo  ...                                              title  \
0        2        0  ...  NASA's Grand Chal

D:\Program_files\Anaconda 3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [153]:
s = "Jinyu Chang"
s.find(',')

-1

In [173]:
dd = pd.read_csv('./train.csv')
print(dd.head(5))
soup = BeautifulSoup(dd.loc[1]['Page content'], "html.parser")
print(soup.prettify())

   Id  Popularity                                       Page content
0   0          -1  <html><head><div class="article-info"> <span c...
1   1           1  <html><head><div class="article-info"><span cl...
2   2           1  <html><head><div class="article-info"><span cl...
3   3          -1  <html><head><div class="article-info"><span cl...
4   4          -1  <html><head><div class="article-info"><span cl...
<html>
 <head>
  <div class="article-info">
   <span class="byline">
    <a href="/author/christina/">
     <img alt="2016%2f06%2f28%2fa2%2fhttpsd2mhye01h4nj2n.cloudfront.netmediazgkymde1lza0.6e864" class="author_image" src="http://i.amz.mshcdn.com/EOF74clpv5lKQmem8EbdqygnSIA=/90x90/2016%2F06%2F28%2Fa2%2Fhttpsd2mhye01h4nj2n.cloudfront.netmediaZgkyMDE1LzA0.6e864.jpg"/>
    </a>
    <span class="author_name">
     By
     <a href="/author/christina/">
      Christina Warren
     </a>
    </span>
    <time datetime="Thu, 28 Mar 2013 17:40:55 +0000">
     2013-03-28 17:40:55 UTC
    